# Linearization of the d-Choquet Integral

In [1]:
# Import basic libraries
from palmerpenguins import load_penguins
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import time
from sklearn.preprocessing import LabelEncoder
import os
os.chdir("/Users/hoangthuyduongvu/Desktop/FuzzSIM")


In [2]:
# Import personalized libraries
from fuzz.src.capacity import *
from fuzz.choquet.choquet import *
from fuzz.src.norm import *
from fuzz.src.knn import KNNFuzz
from fuzz.src.sim import S1, S2, S3
from fuzz.optim import *
from fuzz.utils import *
from fuzz.eval import leave_one_out
from fuzz.dataloader import *
from fuzz.choquet.d_choquet import *

In [3]:
# Load data 
iris_data = load_iris()
iris = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)
iris['target'] = iris_data.target
data, labels = iris.iloc[:, :-1].values, iris.iloc[:, -1].values
data = batch_norm(np.array(data, dtype=np.float32))

data[:5]

array([[1.        , 0.67346936, 0.24489795, 0.        ],
       [1.        , 0.59574461, 0.25531912, 0.        ],
       [1.        , 0.66666669, 0.24444443, 0.        ],
       [1.        , 0.65909088, 0.29545453, 0.        ],
       [1.        , 0.70833325, 0.24999997, 0.        ]])

In [4]:
# Generate all possible mobius for training
features = list(range(len(data[0])))    # encode features as integers

nb_total = 10

# Input Mobius (10 mobius currently)
lst_mobius = []
for i in range(nb_total):
    mobius = generate_mobius(features, 2)
    mobius.pop(0)
    lst_mobius.append(mobius)

# Mutate all mobius
mutated = []
for i in range(len(lst_mobius)):
    mobius = lst_mobius[i]
    mutated_mobius = mutate(mobius, mutation_rate=0.2)
    mutated.append(mutated_mobius)

# Add mutated mobius to the list
lst_mobius.extend(mutated)

# Cross over all mobius
crossed_over = []
for i in range(len(lst_mobius)):
    for j in range(i + 1, len(lst_mobius) // 2):
        crossed = crossover(lst_mobius[i], lst_mobius[j])
        crossed_over.append(crossed)

# Add crossed over mobius to the list
lst_mobius.extend(crossed_over)
print(f"Total number of Möbius measures: {len(lst_mobius)}")

# Convert all Möbius measures to capacities
capacities_list = []
for i in range(len(lst_mobius)):
    mobius = lst_mobius[i]
    capacity = mobius_to_capacity(mobius, features)
    capacities_list.append(capacity)    
print('Test Möbius measures completed!')
print(f"Total number of capacities: {len(capacities_list)}")

Total number of Möbius measures: 65
Test Möbius measures completed!
Total number of capacities: 65


In [5]:
mobius = lst_mobius[0]
print("Möbius:")
for i in range(len(mobius)):
    print(f"Mobius of {mobius[i].X} is {mobius[i].mu:.3f}")

capacity = capacities_list[0]
print("\nDerived Capacity:")
for c in capacity:
    print(f"Capacity of {c.X} is {c.mu:.3f}")


Möbius:
Mobius of [0] is 0.736
Mobius of [1] is 0.760
Mobius of [2] is 0.617
Mobius of [3] is 0.871
Mobius of [0, 1] is 0.078
Mobius of [0, 2] is 0.374
Mobius of [0, 3] is 0.591
Mobius of [1, 2] is 0.198
Mobius of [1, 3] is 0.635
Mobius of [2, 3] is 0.783

Derived Capacity:
Capacity of [] is 0.000
Capacity of [0] is 0.130
Capacity of [1] is 0.135
Capacity of [2] is 0.109
Capacity of [3] is 0.154
Capacity of [0, 1] is 0.279
Capacity of [0, 2] is 0.306
Capacity of [0, 3] is 0.389
Capacity of [1, 2] is 0.279
Capacity of [1, 3] is 0.402
Capacity of [2, 3] is 0.403
Capacity of [0, 1, 2] is 0.490
Capacity of [0, 1, 3] is 0.650
Capacity of [0, 2, 3] is 0.704
Capacity of [1, 2, 3] is 0.685
Capacity of [0, 1, 2, 3] is 1.000


In [9]:
print(restricted_dissim(0.5, 0.3, 1, 0.5))
print(restricted_dissim(0.3, 0.2, 1, 0.5))
print(restricted_dissim(0.2, 0, 1, 0.5))

0.04000000000000001
0.009999999999999995
0.04000000000000001


In [12]:
locate_capacity([1, 2], lst_mobius[0])

0.19835272256864278

In [121]:
def d_choquet_linear_f_x(X: np.ndarray, mobius: List[Capacity], p: float = 1.0, q: float = 1.0, verbose: bool = False):
    """
    Compute the linear d-Choquet integral for a given input and a list of Möbius measures.
    """
    def compute_diss(x: float, X: np.ndarray) -> float:
        s = 0
        X.append(0)
        for i in range(len(X)):
            if X[i] <= x: 
                tmp = [x_p for x_p in X if x_p < X[i]]
                if len(tmp) > 0:
                    s += restricted_dissim(X[i], max(tmp), p, q)
                    # print(f"Restricted dissimilarity between {X[i]} and {max(tmp)}: {restricted_dissim(X[i], max(tmp), p, q)}")
        return s
    sum_result = 0
    for i in range(len(X)):
        sum_result += locate_capacity([i+1], mobius) * compute_diss(X[i], X)
        if verbose:
            print(f"Mobius for {i+1} is {locate_capacity([i+1], mobius)}")
            print(f"Computing dissimilarity for {X[i]} with respect to {X}")
            print(f"Dissimilarity result: {compute_diss(X[i], X):.3f}")
            print(f"Sum result after adding {X[i]}: {sum_result:.3f}")
    return sum_result

In [122]:
mobius = [
    Capacity([], 0),
    Capacity([1], 1/6),
    Capacity([2], 1/6),
    Capacity([3], 1/6),
    Capacity([1, 2], 1/6),
    Capacity([1, 3], 1/6),
    Capacity([2, 3], 1/6),
    Capacity([1, 2, 3], 0)]

In [123]:
d_choquet_linear_f_x([0.5, 0.2, 0.3], mobius, 1, 0.5)

0.03

In [125]:
def d_choquet_linear_g_x(X: np.ndarray, mobius: List[Capacity], p: float = 1.0, q: float = 1.0, verbose: bool = False):
    """
    Compute the linear d-Choquet integral for a given input and a list of Möbius measures.
    """
    def compute_diss(x1: float, x2: float, X: np.ndarray) -> float:
        s = 0
        X.append(0)
        for i in range(len(X)):
            min_x = min(x1, x2)
            if X[i] <= min_x: 
                tmp = [x_p for x_p in X if x_p < X[i]]
                if len(tmp) > 0:
                    s += restricted_dissim(X[i], max(tmp), p, q)

        return s
    sum_result = 0
    for i in range(len(X)):
        for j in range(len(X)):
            if i < j:
                sum_result += locate_capacity([i+1, j+1], mobius) * compute_diss(X[i], X[j], X)
    return sum_result

In [126]:
d_choquet_linear_g_x([0.5, 0.2, 0.3], mobius, 1, 0.5)

0.021666666666666667

In [135]:
def d_choquet_linear(X: np.ndarray, mobius: List[Capacity], p: float = 1.0, q: float = 1.0, verbose: bool = False):
    """
    Compute the linear d-Choquet integral for a given input and a list of Möbius measures.
    
    Parameters:
    - X: Input tensor representing a point in [0,1]^N
    - mobius: List of Möbius measures (capacities)
    - p, q: Parameters for the dissimilarity measure δ_p,q in (0, +∞)
    - verbose: Whether to print intermediate values for debugging
    
    Returns:
    - Linear d-Choquet integral value as a scalar tensor
    """
    if p <= 0 or q <= 0:
        raise ValueError("Parameters p and q must be greater than 0.")
    
    return d_choquet_linear_f_x(X, mobius, p, q, verbose) + d_choquet_linear_g_x(X, mobius, p, q, verbose)

In [136]:
d_choquet_linear([0.5, 0.2, 0.3], mobius, 1, 0.5)

0.051666666666666666